In [9]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:

# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
import more_itertools
from dash.dependencies import Input, Output

import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Launch site drop-down options
launch_sites_options = [{'label':'All sites', 'value':'All sites'}]
launch_sites_list = spacex_df["Launch Site"].unique()
for l in launch_sites_list:
    launch_sites_options.append({'label':l, 'value':l})
    
# Create an app layout
app.layout = html.Div(
    style={'background-color':'#e6f2ff'},
    children=[
        html.H1('SpaceX Launch Records Dashboard',
                style={
                    'textAlign': 'center',
                    'color':'#503D36', 'font-size': 40}),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites
        # dcc.Dropdown(id='site-dropdown',...)
        html.Br(),
        html.Div([
            html.Label("Select Launch Site",
                       style={'color':'#503D36', 'font-size':20}),
            html.Div(dcc.Dropdown(
                id='site-dropdown',
                options=launch_sites_options,
                placeholder="All sites",
                style={'width':'80%'}
            ))]),
      
            # TASK 2: Add a pie chart to show the total successful launches count for all sites
            # If a specific launch site was selected, show the Success vs. Failed counts for the site
            html.Div(id='success-pie-chart'),
            html.Br(),
    
            
            # TASK 3: Add a slider to select payload range
            #dcc.RangeSlider(id='payload-slider',...)
            html.P("Payload range (Kg):"),
            html.Div(
                dcc.Slider(min_payload, max_payload, 1000,
                   value=10,
                   id='my-slider'
            )),
        
            # TASK 4: Add a scatter chart to show the correlation between
            # payload and launch success
            html.Div(id='success-payload-scatter-chart'),
            ])

# TASK 2:
# Add a callback function for `site-dropdown` as input,
# `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='children'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(input_value):
    if input_value == "All sites" or input_value == None:
        df = spacex_df[spacex_df['class'] == 1]
        df = df.groupby("Launch Site").count()
        df.reset_index(inplace=True)
        chart = dcc.Graph(figure=px.pie(df,
                                        values='class',
                                        names='Launch Site',
                                        title='Successful launches for all site'))
    else:
        df = spacex_df[spacex_df["Launch Site"] == input_value]
        df = df.groupby("class").count()
        df.reset_index(inplace=True)
        chart = dcc.Graph(figure=px.pie(df,
                                        values='Unnamed: 0',
                                        names='class',
                                        title='Launches for '+input_value))
    return [
        html.Div(
                className='chart-item',
                children=[html.Div(children=[chart])])
        ]
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs,
# `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart',
           component_property='children'),
    Input(component_id='site-dropdown', component_property='value')
)
def success_payload_chart(input_value):
    chart = dcc.Graph(figure = px.scatter(spacex_df,
                 x="Payload Mass (kg)",
                 y="class",
                 color="Launch Site"))
    return [
        html.Div(
                className='chart-item',
                children=[html.Div(children=[chart])])
        ]

# Run the app
if __name__ == '__main__':
    app.run_server(port=8051)
